## Exploratory Data Analysis (EDA)

Objective:
    1. To get an insight into input dataframe.
    2. To get an understanding of basic statistics.
    3. Identify features of importance through VIF, PCA and / or Decision Trees

Assumptions:
    1. Only *.csv files are currently being read.
    2. Target columns are placed after the features in the *.csv
    3. Median imputing is performed where needed. Change it appropriately as needed.

#### Libraries

In [ ]:
# Import relevant libraries

import os

import warnings
warnings.filterwarnings("ignore")

import numpy as np

import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns

import pandas as pd
pd.set_option('display.max_columns', 100)
pd.options.display.width=None

import ppscore as pps

from tabulate import tabulate
tabulate.PRESERVE_WHITESPACE = False

from pca import pca
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.decomposition import PCA as SKLPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#### Global Constants

In [ ]:
# Global constants

RND_STATE = 39  # random_state where used is assigned RND_STATE
TESTSIZE = 0.2  # test_size where used is assigned TESTSIZE
huecol = ['Dataset'] # Recommended for using with Classification Dataset. If not used, must specify None

#### I/O Path

In [ ]:
PATH = r"C:\DSML_Case_Studies\01_Classification\01_Dataset"
OUTPATH = r"C:\DSML_Case_Studies\01_Classification\03_Output"
DATASET = r"\Dataset_Indian_Liver_Patient_10x1.csv"
PREFIX = r"\GS_EDA_V1.2_IndianLiverPatient_" # Version#_DatasetName_FullFeatureSet_NoOutlierRemoval__StratifyY_Upsampling_StandardScaler_Metric_Seed_Iteration

# Specify number of features and targets

n_features = 10
n_target = 1

#### Dataframe Construction

In [ ]:
# Dataframe Definition & Classifying Features & Targets

df = pd.read_csv(f"{PATH}{DATASET}")
df = df.round(decimals=3)

DF_INFO = df.dtypes.to_frame('Data Type').reset_index()

collst = []
for columns in df.columns:
    collst.append(columns)

featlst = collst[0:len(collst)-n_target]
targlst = collst[-n_target:]

cat_df = df.select_dtypes(include=['object'])
catlst = []
for col in cat_df.columns:
    catlst.append(col)

y_catlst = [value for value in catlst if value in targlst]

In [ ]:
print("Dataframe BEFORE Encoding: ")

In [ ]:
df.head()

#### Encoding Categorical Feature

In [ ]:
catfeatlst = [value for value in catlst if value not in targlst]

for i in range(0, len(catfeatlst), 1):
    colprefix = catfeatlst[i]
    df = pd.get_dummies(df, prefix=[catfeatlst[i]], columns=[catfeatlst[i]], drop_first=True)

#### Encoding Categorical Target

In [ ]:
# Encode Categorical Columns

for i in range(0, len(y_catlst), 1):
    df[y_catlst[i]] = df[y_catlst[i]].astype('category')
    df[y_catlst[i]] = df[y_catlst[i]].cat.codes
    continue

In [ ]:
print("Dataframe AFTER Encoding: ")

In [ ]:
df.head()

In [ ]:
print("Dataframe BEFORE Imputing: ")

In [ ]:
print(df.isnull().sum(), end='\n\n')

#### Missing Data Imputation

In [ ]:
# Basic Imputing

''' Use Appropriate Imputer - Mean, Meadian, Mode... Others '''

df = df.apply(lambda x: x.fillna(x.median()), axis=0)


In [ ]:
print("Dataframe AFTER MEDIAN Imputing: ")

In [ ]:
print(df.isnull().sum(), end='\n\n')

In [ ]:
# Sanity checks before proceeding further

print("List of Features:", featlst, end='\n\n')
print("List of Targets:", targlst, end='\n\n')
print("List of Categorical Variables:", catlst, end='\n\n')
print("List of Categorical Targets", y_catlst, end='\n\n')

#### Descriptive Statistics

In [ ]:
print("Descriptive Stats: ")

In [ ]:
desc_stat = df.describe().T.round(3) # Univariate analyses
print(tabulate(desc_stat, headers=desc_stat.columns, tablefmt="github", numalign="right"))

#### Count, Q-Q & Box Plots

In [ ]:
# Check for Normality - Visual Check - Plots not being saved.

collst = []
for columns in df.columns:
    collst.append(columns)

for x in collst:
    fig,ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
    sns.histplot(data=df, x=x, kde=True, hue=huecol[0], ax=ax[0])
    sm.qqplot(df[x],ax=ax[1],line='45',fit=True)
    ax[1].set_xlabel(x)
    sns.boxplot(data=df,x=x,ax=ax[2])
    sns.stripplot(data=df,x=x,ax=ax[2], color='green')
    plt.show()

#### Feature vs Target Box Plot

In [ ]:
# Box Plot

if len(y_catlst)!= 0:
    NCOLS = 4
    m_rows = int(np.ceil((len(collst)-len(y_catlst))/NCOLS))
    fig, axes = plt.subplots(m_rows, NCOLS, figsize = (15,15))
    axes = axes.flatten()
    for lst in range(0, len(y_catlst), 1):
        temp = 'Fig_0' + str(lst)
        FIG1 = f"{temp}_Boxplot"
        for i in range(0,len(df.columns)-len(y_catlst)):
            sns.boxplot(x=y_catlst[lst], y=df.iloc[:,i], data=df, orient='v', ax=axes[i])
            plt.tight_layout()
if len(y_catlst)== 0:
    FIG1 = r"Fig_01_Boxplot"
    lst = [x for x in collst if x not in y_catlst]
    fig, axes = plt.subplots(1, len(lst), figsize = (45,15))
    axes = axes
    for i, col in enumerate(lst):
        ax = sns.boxplot(y=df[col], ax=axes.flatten()[i])
        axminlt = df[col].min()-0.1*df[col].min()
        axmaxlt = df[col].max()+0.1*df[col].max()
        ax.set_ylim(axminlt, axmaxlt)
        ax.set_ylabel(col)
    plt.tight_layout()

#### Scatter Plots

In [ ]:
# Scatter Plot - Columns of Interest

sns.pairplot(df, hue=huecol[0])

#### Correlation Heatmaps

In [ ]:
# Linear Correlation Heatmap

cormethod = {0:'pearson', 1:'kendall', 2:'spearman'}
for i in range(0, 3, 1):
    temp = 'linear_cor' + str(i)
    temp = df.corr(method=cormethod[i])
    ftemp = cormethod[i].title()
    FIG2 = r"Fig_02_Corr_"
    mask = np.zeros(temp.shape, dtype=bool)
    mask[np.triu_indices(len(mask))] = True
    plt.subplots(figsize=(20,15))
    plt.title(f"{ftemp} Corrlelation")
    sns.heatmap(temp, annot=True, vmin=-1, vmax=1, center=0,
                cmap='coolwarm', square=True, mask=mask, fmt='.2f')
    plt.show()

#### Non-Linear Correlation Heatmaps

In [ ]:
# Non-Linear Correlation Predictive Power Score - Heatmap

FIG3 = r"Fig_03_Predictive_Power_Score"

ppscorr = pps.matrix(df) # Predictive Power Score - PPS
matrix_df = pps.matrix(df)[['x', 'y', 'ppscore']].pivot(columns='x', index='y',
                                                        values='ppscore')
plt.subplots(figsize=(20,15))
sns.heatmap(matrix_df, cmap="coolwarm", annot=True, linewidth=0, annot_kws={"size":12}, fmt='.2f')
plt.show()

#### VIF

In [ ]:
# Feature Reduction -  Variance Inflation Factor [VIF]

vif_data = pd.DataFrame()
vif_data["Feature"] = df.columns
vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(len(df.columns))]

In [ ]:
print("------ Variance Inflation Factor ------")

In [ ]:
vif_data.head(len(collst)).round(decimals=2)

#### Feature Scaling

In [ ]:
if n_target == 0:
    X = df.copy()
else:
    X = df.drop(columns=targlst)
    y = df.filter(targlst, axis=1)

scaler = StandardScaler()
# scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)

#### Principal Component Analyses

In [ ]:
# Principal Component Analyses

sklpca = SKLPCA(n_components=X.shape[1], svd_solver='full', random_state=RND_STATE)
sklpca.fit(X_scaled)
X_transform = sklpca.transform(X_scaled)

pricom = pd.DataFrame(sklpca.components_.round(3)) # Principal Components
pricomvar = pd.DataFrame(sklpca.explained_variance_ratio_.round(3))

n_pcs = pricom.shape[0] # Number of Components
topfeat = [np.abs(sklpca.components_[i]).argmax() for i in range(n_pcs)]

# Get the Feature Names
featnames = []
for columns in X.columns:
    featnames.append(columns)

topfeatname = [featnames[topfeat[i]] for i in range(n_pcs)]
dic = {'PC{}'.format(i+1): topfeatname[i] for i in range(n_pcs)}

# Build the dataframe
df_topfeat = pd.DataFrame((dic.items()))
n_pca_comp = sklpca.n_components_
print("No. of Components Explaining 95% Variance:", n_pca_comp)

In [ ]:
# Top 10 Features from PCA

In [ ]:
df_topfeat.head(10)

In [ ]:
# EDA Report Out

# Output to Excel

SUMMARY = r"Summary.xlsx"

writer = pd.ExcelWriter(f"{OUTPATH}{PREFIX}{SUMMARY}", engine='xlsxwriter', options={'strings_to_numbers': True})
DF_INFO.to_excel(writer, sheet_name='Info')
desc_stat.to_excel(writer, sheet_name='Stats')
vif_data.to_excel(writer, sheet_name='VIF')
pricomvar.to_excel(writer, sheet_name='PCA_VAR')
pricom.to_excel(writer, sheet_name='PCA_Components')
df_topfeat.to_excel(writer, sheet_name='PCA_Top_Features')
writer.save()

#### Notbook to HTML

In [ ]:
# !jupyter nbconvert --output-dir="C:\DSML_Case_Studies\01_Classification\00_Final_Reports" GS_EDA_V1.1.ipynb --to html --no-input

In [ ]:
NB_Convert = u" jupyter nbconvert --to html_toc --output " # Trailing space is required.
NB_Name = u"GS_EDA_V1.2.ipynb"
PREFIX_A = PREFIX[:-1]
HTML_Name = PREFIX_A.replace("\\", "") + " " # Trailing space is required.
Names_Concat = NB_Convert + HTML_Name + NB_Name
try:
    if(__IPYTHON__):
        get_ipython().system(Names_Concat)
except NameError:
    pass